In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, roc_auc_score
import utils
import plotting
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import mplhep as hep
import vector

In [2]:
#Load your dataset
samples = {
    "hh4b": ["GluGlutoHHto4B_cHHH1_TuneCP5_PSWeights_13TeV-powheg-pythia8"],
    # "hh4b-c2p45": ["GluGlutoHHto4B_cHHH2p45_TuneCP5_PSWeights_13TeV-powheg-pythia8"],
    # "hh4b-c5": ["GluGlutoHHto4B_cHHH5_TuneCP5_PSWeights_13TeV-powheg-pythia8"],
    # "hh4b-c0": ["GluGlutoHHto4B_cHHH0_TuneCP5_PSWeights_13TeV-powheg-pythia8"],
    "qcd": [
        "QCD_HT-200to300-13TeV",
        "QCD_HT-300to500-13TeV",
        "QCD_HT-500to700-13TeV",
        "QCD_HT-700to1000-13TeV",
        "QCD_HT-1000to1500-13TeV",
        "QCD_HT-1500to2000-13TeV",
        "QCD_HT-2000toInf-13TeV",
    ],
    "ttbar": [
        "TTTo2L2Nu_13TeV",
        "TTToHadronic_13TeV",
        "TTToSemiLeptonic_13TeV",
        
    ],    
}

year = "2018"#
dir_name = "23Nov18_WSel_v9_private"
path_to_dir = f"/eos/uscms/store/user/cmantill/bbbb/matching/{dir_name}/"
dirs = {path_to_dir: samples}

filters = [
    [
        # one good fatjet (fatjet with index 0 has the largest Xbb score)
        ("('ak8FatJetPt', '0')", ">=", 300),
        ("('ak8FatJetMsd', '0')", ">=", 60),
        ("('ak8FatJetPNetXbb', '0')", ">=", 0.9734),  # medium WP
        # second fatjet has lower
        ("('ak8FatJetPNetXbb', '1')", "<", 0.9880),  # tight WP
    ]
]

# columns to load                                                                                                                                                                                                                                     
# the parquet files are too big so we can only load a few columns at a time without consumming much memory
load_columns = [
    ("weight", 1),
	("ak8FatJetMsd", 2),
    ("ak8FatJetPNetMass", 2),
	("ak8FatJetPNetXbb", 2),                                                                                                                                                                                                                     
]
# reformat into ("column name", "idx") format for reading multiindex columns                                                                                                                                                                          
columns = []
for key, num_columns in load_columns:
    for i in range(num_columns):
        columns.append(f"('{key}', '{i}')")


events_dict = {}
for input_dir, samples in dirs.items():
    events_dict = {
        **events_dict,
        # this function will load files (only the columns selected), apply filters and compute a weight per event
        **utils.load_samples(input_dir, samples, year, filters=filters),
    }

samples_loaded = list(events_dict.keys())
keys_loaded = list(events_dict[samples_loaded[0]].keys())

#This section prints all keys for inspection
#print(f"Keys in events_dict")
#for i in keys_loaded:
#    print(i)


# Assuming df has features and a binary target column named 'target'
#features = df.drop('target', axis=1)
#target = df['target']

Loading GluGlutoHHto4B_cHHH1_TuneCP5_PSWeights_13TeV-powheg-pythia8
/eos/uscms/store/user/cmantill/bbbb/matching/23Nov18_WSel_v9_private//2018/GluGlutoHHto4B_cHHH1_TuneCP5_PSWeights_13TeV-powheg-pythia8/parquet
Loading QCD_HT-1000to1500-13TeV
/eos/uscms/store/user/cmantill/bbbb/matching/23Nov18_WSel_v9_private//2018/QCD_HT-1000to1500-13TeV/parquet
Loading QCD_HT-1500to2000-13TeV
/eos/uscms/store/user/cmantill/bbbb/matching/23Nov18_WSel_v9_private//2018/QCD_HT-1500to2000-13TeV/parquet
Loading QCD_HT-2000toInf-13TeV
/eos/uscms/store/user/cmantill/bbbb/matching/23Nov18_WSel_v9_private//2018/QCD_HT-2000toInf-13TeV/parquet
Loading QCD_HT-200to300-13TeV
/eos/uscms/store/user/cmantill/bbbb/matching/23Nov18_WSel_v9_private//2018/QCD_HT-200to300-13TeV/parquet
Loading QCD_HT-300to500-13TeV
/eos/uscms/store/user/cmantill/bbbb/matching/23Nov18_WSel_v9_private//2018/QCD_HT-300to500-13TeV/parquet
Loading QCD_HT-500to700-13TeV
/eos/uscms/store/user/cmantill/bbbb/matching/23Nov18_WSel_v9_private//2018

In [3]:
def make_vector(events: pd.DataFrame, obj: str):
    """Create a ``vector`` object from the columns of the dataframe"""
    mstring = "PNetMass" if obj == "ak8FatJet" else "Mass"

    return vector.array(
        {
            "pt": events[f"{obj}Pt"],
            "phi": events[f"{obj}Phi"],
            "eta": events[f"{obj}Eta"],
            "M": events[f"{obj}{mstring}"],
        }
    )

In [4]:
#DEFINE FEATURES & TARGET

# Higgs candidate selection example
events = pd.concat([events_dict["hh4b"], events_dict["qcd"], events_dict["ttbar"]], keys=['hh4b','qcd', 'ttbar'])

events['target'] = 0  # Default to 0 (background)
events.loc['hh4b', 'target'] = 1  # Set to 1 for 'hh4b' samples (signal)

#print(events)

# jets outside the fatjet - sorted by b-score
jets_outside = make_vector(events, "ak4JetOutside")
# fatjets sorted by xbb
fatjets = make_vector(events, "ak8FatJet")

# H1 candidate
h1 = fatjets[:, 0]
h1_xbb = events.ak8FatJetPNetXbb[0]

# H2 candidate
# ak4 jet (outside) with highest b-tagging score (btagDeepFlavB)
j3 = jets_outside[:, 0]
j4 = jets_outside[:, 1]
j3_btag = events.ak4JetOutsidebtagDeepFlavB[0]

print(j3_btag)
j4_btag = events.ak4JetOutsidebtagDeepFlavB[1]
h2 = j3 + j4
hh = h1 + h2

# Second fatjet
fj2 = fatjets[:, 1]
fj2_xbb = events.ak8FatJetPNetXbb[1]



df = pd.DataFrame()




#df['ak4JetOutsideEta'] = events_dict['ak4GenJetEta', 0]
#df['ak8FatJetPNetMass'] = events.ak8FatJetPNetMass[0]

#TODO: remove h1_xbb, hb_mass from dataframe, this currently causes df indexing issues

#df['h1_xbb'] = h1_xbb#
df['logPtHH'] = np.log(hh.pt)#
df['logPtH1'] = np.log(h1.pt)#
df['logPtJ3'] = np.log(j3.pt)#
df['logPtJ4'] = np.log(j4.pt)#
df['H1Pt_H2Pt'] = h1.pt#
df['dPhiJ3J4'] = j3.deltaphi(j4)
df['dRJ3J4'] = j3.deltaR(j4)#
df['j3_btag'] = j3_btag#
df['j4_btag'] = j4_btag#
df['j3_j4_ratio'] = j3.pt/j4.pt#
df['hb_eta'] = h1.eta#
#df['hb_mass'] = h1.mass#
df['hr_pt'] = h2.pt#
df['hr_eta'] = h2.eta#
df['hr_mass'] = h2.mass#
hh.eta[np.isinf(hh.eta)] = 10000
df['HHEta'] = hh.eta#  # this triggers invalid value exception due to inf values
df['HHMass'] = hh.M#
df['fj2_pt'] = fj2.pt#
df['fj2_eta'] = fj2.eta#
df['fj2_mass'] = fj2.M#
df['fj2_btag'] = fj2_xbb#
drh1h2 = np.sqrt((h1.eta-h2.eta)**2 + (h1.phi - h2.phi)**2)
drh1h2[np.isinf(drh1h2)] = 10000
df['dRH1H2'] = drh1h2#
df['dphi'] = h1.deltaphi(h2)#
df['deta'] = h1.deltaeta(h2)#

df.replace(np.inf, 10000, inplace=True)
df.replace(-np.inf, 10000, inplace=True)
#print(df['ak8FatJetPNetMass'])



/tmp/ipykernel_25843/1199893880.py:7: PerformanceWarning: indexing past lexsort depth may impact performance.
  events.loc['hh4b', 'target'] = 1  # Set to 1 for 'hh4b' samples (signal)


hh4b   0         0.996094
       1         0.999512
       2         0.959961
       3         0.992676
       4         0.997070
                   ...   
ttbar  194517    0.311035
       194518    0.899902
       194519    0.989258
       194520    0.999512
       194521    0.999023
Name: 0, Length: 502802, dtype: float64


/tmp/ipykernel_25843/1199893880.py:48: RuntimeWarning: invalid value encountered in log
  df['logPtJ3'] = np.log(j3.pt)#
/tmp/ipykernel_25843/1199893880.py:49: RuntimeWarning: invalid value encountered in log
  df['logPtJ4'] = np.log(j4.pt)#


ValueError: cannot handle a non-unique multi-index!

In [ ]:
#LOAD FEATURES INTO PD DF

features = df

#define target
target = events['target']

In [ ]:
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

In [ ]:
# Initialize the BDT model
bdt_model = XGBClassifier(
    n_estimators= 196,
    max_depth= 17,
    learning_rate=0.1,
    subsample=0.5,
    random_state=42
)



In [ ]:
# Train the model
bdt_model.fit(X_train, y_train)

In [ ]:
# Make predictions
predictions = bdt_model.predict(X_test)


In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test, predictions)
roc_auc = roc_auc_score(y_test, predictions)

In [ ]:
print(predictions)

In [ ]:
probabilities = bdt_model.predict_proba(X_test)

print(probabilities[:,0])

In [ ]:
y_scores = bdt_model.predict_proba(X_test)[:, 1]

fpr, tpr, thresholds = roc_curve(y_test, y_scores)
roc_auc = auc(fpr, tpr)

#histogram for y scores signal & background 
#rank by importance

#number of b-jets that can be identified in resolved
#another feather cos(theta_star)

#Plottting
plt.figure()
plt.plot(tpr, fpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.xlabel('Signal (HH)')
plt.ylabel('Background (QCD&ttbar)')
plt.title('ROC')
plt.legend(loc="upper left")
plt.yscale('log')
plt.savefig('plots/ROC_BDT.png')
plt.show()


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from skopt import BayesSearchCV
import numpy as np

# Load data
iris = load_iris()
X, y = iris.data, iris.target

# Define the model
model = RandomForestClassifier()

# Define the hyperparameters and their ranges
search_space = {
    'n_estimators': (10, 200),  # Integer range
    'max_depth': (5, 50)        # Integer range
}

# Set up Bayesian optimization
bayes_search = BayesSearchCV(model, search_space, n_iter=32, scoring='accuracy', cv=5, n_jobs=-1, random_state=0)

# Perform the Bayesian optimization
bayes_search.fit(X, y)

# Best parameters
print("Best parameters:", bayes_search.best_params_)


In [ ]:
#determine importance of the features

importances = bdt_model.feature_importances_

feature_names = df.columns
feature_importance = sorted(zip(feature_names, importances), key=lambda x: x[1], reverse=True)

for feature in feature_importance:
    print(f"{feature[0]}: {feature[1]:.3f}")


In [ ]:
hh4b_scores = bdt_model.predict_proba(X_test.loc['hh4b'])[:, 1]
qcd_scores = bdt_model.predict_proba(X_test.loc['qcd'])[:, 1]
ttbar_scores = bdt_model.predict_proba(X_test.loc['ttbar'])[:, 1]

In [ ]:
plt.figure()
plt.hist(hh4b_scores, bins=40, histtype='step', linewidth=1.5, color='darkblue')
plt.hist(qcd_scores, bins=40, histtype='step', linewidth=1.5, color='red')
plt.hist(ttbar_scores, bins=40, histtype='step', linewidth=1.5, color='darkgreen')
plt.legend(['hh4b', 'qcd', 'ttbar'])
plt.ylabel('Frequency')
plt.xlabel('Prediciton Score')
plt.title('Model Predictions Histogram')
plt.yscale('log')
plt.show()
